In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
data = pd.read_csv('/content/drive/MyDrive/cnn_proj_data/dataset/archive_data/ckextended.csv')

In [3]:
print(data.head())
print(data.dtypes)

   emotion                                             pixels     Usage
0        6  36 39 35 25 19 11 8 7 3 13 15 9 21 57 75 90 10...  Training
1        6  88 74 19 4 5 5 3 12 8 21 15 21 15 18 24 29 32 ...  Training
2        6  9 2 4 7 1 1 1 0 7 29 49 76 115 141 156 169 177...  Training
3        6  104 106 108 104 95 50 60 61 58 83 126 133 139 ...  Training
4        6  68 72 67 67 6 2 1 1 1 1 1 14 24 24 38 65 79 94...  Training
emotion     int64
pixels     object
Usage      object
dtype: object


In [4]:
def process_pixels(pixels):
    if isinstance(pixels, str):
        pixels = np.array(pixels.split(), dtype='float32')
    return pixels / 255.0

data['pixels'] = data['pixels'].apply(process_pixels)

# Extract features and labels
X = np.stack(data['pixels'].values)
y = data['emotion'].values

# Check unique values in 'emotion' column
print(f"Unique emotion labels: {np.unique(y)}")

# Reshape the data
X = X.reshape(X.shape[0], 48, 48, 1)  #images are 48x48 pixels

# One-hot encode labels
from keras.utils import to_categorical
num_classes = 8
y = to_categorical(y, num_classes=num_classes)

# Split the data into training and testing sets
train_indices = data['Usage'] == 'Training'
test_indices = data['Usage'] == 'PublicTest'

X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]


print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Unique emotion labels: [0 1 2 3 4 5 6 7]
Training set size: 734
Test set size: 91


In [15]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))


In [17]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
batch_size = 32
epochs = 75

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=1)


Epoch 1/75
23/23 [==============================] - 3s 128ms/step - loss: 1.0124 - accuracy: 0.7575 - val_loss: 1.0862 - val_accuracy: 0.7582
Epoch 2/75
23/23 [==============================] - 2s 102ms/step - loss: 1.0301 - accuracy: 0.7589 - val_loss: 1.0728 - val_accuracy: 0.7582
Epoch 3/75
23/23 [==============================] - 2s 105ms/step - loss: 1.0149 - accuracy: 0.7575 - val_loss: 1.0571 - val_accuracy: 0.7802
Epoch 4/75
23/23 [==============================] - 4s 177ms/step - loss: 1.0093 - accuracy: 0.7657 - val_loss: 1.0602 - val_accuracy: 0.7473
Epoch 5/75
23/23 [==============================] - 3s 110ms/step - loss: 0.9702 - accuracy: 0.7589 - val_loss: 1.0544 - val_accuracy: 0.7582
Epoch 6/75
23/23 [==============================] - 2s 103ms/step - loss: 0.9679 - accuracy: 0.7711 - val_loss: 1.0305 - val_accuracy: 0.7802
Epoch 7/75
23/23 [==============================] - 2s 100ms/step - loss: 0.9336 - accuracy: 0.7793 - val_loss: 1.0238 - val_accuracy: 0.8022
Epoch 

In [20]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Test loss: 0.8128833174705505
Test accuracy: 0.8681318759918213
